In [8]:
import os
import time
import glob
import itertools as it
import numpy as np
import cv2
import pickle
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label
from moviepy.editor import VideoFileClip
%run Functions.py

In [12]:
#load data from trainer
cls_dict = pickle.load(open("classifier_LinearSVC_YCrCb_991_HogALL.p", "rb"))

# Pull in parameters and classifier from dictionary
color_space = cls_dict['color_space']
orient = cls_dict['orient']
pix_per_cell = cls_dict['pix_per_cell']
cell_per_block = cls_dict['cell_per_block']
hog_channel = cls_dict['hog_channel']
spatial_size = cls_dict['spatial_size']
hist_bins = cls_dict['hist_bins']
spatial_feat = cls_dict['spatial_feat']
hist_feat = cls_dict['hist_feat']
hog_feat = cls_dict['hog_feat']
y_start_stop = cls_dict['y_start_stop']
ystart = y_start_stop[0]
ystop = y_start_stop[1]
svc = cls_dict['svc']
X_scaler = cls_dict['X_scaler']
print(cls_dict)
#scale = 1.5
scale = [1.1, 1.5, 1.9, 2.3]
heat_threshold = 1
heat_smooth_factor = 0
heat_smooth_factor = 0.5
xstart=600
heat_list=[]

{'svc': LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0), 'orient': 32, 'pix_per_cell': 16, 'hist_bins': 32, 'hist_feat': True, 'hog_channel': 'ALL', 'spatial_size': (32, 32), 'spatial_feat': True, 'y_start_stop': [400, 656], 'cell_per_block': 2, 'X_scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'hog_feat': True, 'color_space': 'YCrCb'}


In [13]:
global heat_list
def pipeline(img):
    global heat_list
    scale = 1.5
    _, heat_map1, img_boxes1, _ = find_cars(img, ystart, 600, xstart, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space=color_space)
    scale = 2
    _, heat_map2, img_boxes2, _ = find_cars(img, 500, ystop, xstart, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins, color_space=color_space)
    
    hot_windows = img_boxes1+img_boxes2
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat = add_heat(heat,hot_windows)
    heat_thresh=heat
    if len(heat_list)==0:
        heat_comb = np.zeros_like(img[:,:,0]).astype(np.float)
        heat_comb_thresh = heat_comb
        labels = label(heat_thresh)
        heat_list = heat_thresh
    else:
        heat_comb = heat_list*heat_smooth_factor + heat_thresh*(1-heat_smooth_factor)
        heat_comb_thresh = apply_threshold(heat_comb,heat_threshold)
        labels = label(heat_comb_thresh)
        heat_list = heat_comb_thresh
    draw_img = draw_labeled_bboxes(img, labels)
    
    return draw_labeled_bboxes(img, labels)

In [14]:
heat_list=[]
project_output = 'project_video_output_alternate3.mp4'
clip1 = VideoFileClip("project_video - Copy.mp4")
project_clip = clip1.fl_image(pipeline)
%time project_clip.write_videofile(project_output, audio=False)

[MoviePy] >>>> Building video project_video_output_alternate3.mp4
[MoviePy] Writing video project_video_output_alternate3.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [04:34<00:00,  4.73it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output_alternate3.mp4 

Wall time: 4min 34s
